<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/MAsk_FActory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**"Fabbrica di Maschere" (Mask Factory)**. Un singolo script che:
1.  Prende in input tutte le 354 patch che hai creato.
2.  Applica una serie di filtri e soglie intelligenti per identificare automaticamente le tre regioni: **Sfondo, Foro, Danno**.
3.  "Pulisce" il risultato per rimuovere il rumore.
4.  Salva una maschera numerica (0, 1, 2) per ogni patch, pronta per essere data in pasto alla U-Net.


In [ ]:
!file /content/Radio_Patches_Output.zip

In [ ]:
!unzip "/content/Radio_Patches_Output"

In [ ]:
!pip install opencv-python
!pip install tqdm

TEST parametri

In [ ]:
#### La Logica della "Mask Factory v5" (Il Piano Definitivo)
##
##1.  **Trova il Foro Principale:** Usa `HoughCircles` sull'immagine originale per trovare il cerchio del foro. Questo ci dà un centro e un raggio "di riferimento". Questa è la nostra ancora geometrica.
##2.  **Crea una Maschera di Interesse per il Foro:** Disegna un cerchio pieno basato sul riferimento trovato. Considereremo "foro" (Classe 1) solo i pixel che sono molto chiari (`>THRESHOLD_FORO`) **E** cadono all'interno di questo cerchio. Questo elimina immediatamente la freccia e i bordi.
##3.  **Crea una Maschera di Interesse per il Danno:** Prende il cerchio di riferimento e lo "ingrassa" (`dilate`) di `ROI_DILATION_PIXELS`. Questa è la nostra "ciambella" di ricerca del danno.
##4.  **Trova il Danno:** Considereremo "danno" (Classe 2) solo i pixel che sono molto scuri (`<THRESHOLD_DANNO`) **E** cadono all'interno della "ciambella".
##5.  **Pulizia Finale:** Applica la pulizia dei piccoli blob solo sul danno trovato, per eliminare il rumore residuo.
##6.  **Assemblaggio:** Unisce la maschera del foro pulita e la maschera del danno pulita.
##
##Questo approccio è top-down: prima definisce la geometria e poi la usa per qualificare i pixel. È molto più robusto di un approccio bottom-up (qualifica i pixel e poi cerca di capire la geometria).
##
##---
##
##### CELLA UNICA: "Mask Factory v5" con QA
##
##Questo script sostituisce tutto il processo precedente. È autonomo e produce l'output finale (maschere e QA) in un colpo solo.
##
##```python
# MASK FACTORY v5 - APPROCCIO TOP-DOWN CON GEOMETRIA
import cv2
import numpy as np
import os
from pathlib import Path
from tqdm.notebook import tqdm
import shutil

# --- IMPOSTAZIONI GLOBALI E PARAMETRI DI TUNING ---
# Cartella radice con le patch originali
BASE_PATCH_DIR = Path("/content/drive/MyDrive/RadioPatches/Carbon Textile 3B")

# Cartelle di output finali
FINAL_MASK_DIR = Path("/content/Radio_Masks_Final_v5")
FINAL_QA_DIR = Path("/content/Radio_QA_Final_v5")

# Parametri per i pixel
THRESHOLD_DANNO = 45
THRESHOLD_FORO = 240

# Parametri per la geometria del FORO
HOUGH_DP = 1.2
HOUGH_MINDIST = 400
HOUGH_PARAM1 = 50
HOUGH_PARAM2 = 25
HOUGH_MINRADIUS = 135
HOUGH_MAXRADIUS = 150

# Parametri per la geometria del DANNO
# Definisce lo spessore della "ciambella" di ricerca attorno al foro
ROI_DILATION_PIXELS = 48

# Parametri per la pulizia finale
MIN_AREA_PULIZIA_DANNO = 44

# --- SCRIPT ---
# Pulizia delle cartelle di output per un'esecuzione pulita
for d in [FINAL_MASK_DIR, FINAL_QA_DIR]:
    if d.exists():
        shutil.rmtree(d)
    d.mkdir(exist_ok=True, parents=True)

all_patch_files = sorted(BASE_PATCH_DIR.glob("**/*.jpg"))
print(f"Trovate {len(all_patch_files)} patch totali da processare.")

for patch_path in tqdm(all_patch_files, desc="Creo maschere v5..."):
    img_gray = cv2.imread(str(patch_path), cv2.IMREAD_GRAYSCALE)
    if img_gray is None: continue

    # --- 1. TROVA IL RIFERIMENTO GEOMETRICO (IL CERCHIO DEL FORO) ---
    # Usiamo una versione sfocata dell'immagine per rendere Hough più robusto
    blurred_img = cv2.GaussianBlur(img_gray, (5, 5), 0)
    circles = cv2.HoughCircles(blurred_img, cv2.HOUGH_GRADIENT, dp=HOUGH_DP, minDist=HOUGH_MINDIST,
                               param1=HOUGH_PARAM1, param2=HOUGH_PARAM2, minRadius=HOUGH_MINRADIUS, maxRadius=HOUGH_MAXRADIUS)

    if circles is None:
        print(f"⚠️ NESSUN CERCHIO TROVATO IN: {patch_path.name}. Salto.")
        continue

    # Prendi il cerchio più forte (di solito l'unico)
    c = np.uint16(np.around(circles))[0, 0]
    center, radius = (c[0], c[1]), c[2]

    # --- 2. ISOLA IL FORO USANDO LA GEOMETRIA ---
    # Crea la maschera del cerchio ideale
    foro_geom_mask = np.zeros_like(img_gray)
    cv2.circle(foro_geom_mask, center, radius, 255, -1)

    # Trova i pixel chiari
    _, foro_pixel_mask = cv2.threshold(img_gray, THRESHOLD_FORO, 255, cv2.THRESH_BINARY)

    # Il vero foro è l'intersezione tra la geometria e i pixel chiari
    foro_finale_mask = cv2.bitwise_and(foro_pixel_mask, foro_geom_mask)

    # --- 3. ISOLA IL DANNO USANDO LA GEOMETRIA ---
    # Crea la "ciambella" di interesse per il danno
    roi_kernel = np.ones((ROI_DILATION_PIXELS, ROI_DILATION_PIXELS), np.uint8)
    danno_geom_mask = cv2.dilate(foro_geom_mask, roi_kernel, iterations=1)

    # Trova i pixel scuri
    _, danno_pixel_mask = cv2.threshold(img_gray, THRESHOLD_DANNO, 255, cv2.THRESH_BINARY_INV)

    # Il danno potenziale è l'intersezione tra la ciambella e i pixel scuri
    danno_potenziale_mask = cv2.bitwise_and(danno_pixel_mask, danno_geom_mask)

    # --- 4. PULIZIA FINALE DEL DANNO ---
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(danno_potenziale_mask, connectivity=8)
    danno_pulito_mask = np.zeros_like(danno_potenziale_mask)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] >= MIN_AREA_PULIZIA_DANNO:
            danno_pulito_mask[labels == i] = 255

    # --- 5. ASSEMBLAGGIO E SALVATAGGIO ---
    final_mask = np.zeros_like(img_gray, dtype=np.uint8)
    final_mask[foro_finale_mask == 255] = 1
    final_mask[danno_pulito_mask == 255] = 2

    # Salva la maschera numerica
    mask_save_path = FINAL_MASK_DIR / patch_path.relative_to(BASE_PATCH_DIR).with_suffix(".png")
    mask_save_path.parent.mkdir(exist_ok=True, parents=True)
    cv2.imwrite(str(mask_save_path), final_mask)

    # Crea e salva il QA
    img_color = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)
    qa_overlay = img_color.copy()
    qa_overlay[final_mask == 1] = [100, 100, 255]
    qa_overlay[final_mask == 2] = [0, 220, 220]
    qa_final = cv2.addWeighted(img_color, 0.6, qa_overlay, 0.4, 0)

    qa_save_path = FINAL_QA_DIR / patch_path.relative_to(BASE_PATCH_DIR)
    qa_save_path.parent.mkdir(exist_ok=True, parents=True)
    cv2.imwrite(str(qa_save_path), qa_final)

print("\n🎉 Processo completato! Controlla i risultati in:")
print(f"✅ Maschere finali: {FINAL_MASK_DIR}")
print(f"✅ Immagini di QA: {FINAL_QA_DIR}")

In [ ]:
#@title visualizza aschere
import cv2, numpy as np, matplotlib.pyplot as plt

mask = cv2.imread("/content/Radio_Masks_Cleaned/H001_h001_Carbon Textile 1A.png", 0)#"/content/unet_dataset_multiclasse/masks/hole_0001.png", 0)
vis  = (mask * 120).astype(np.uint8)   #   0→0, 1→120, 2→240  (solo per VEDERE)
plt.imshow(vis, cmap='inferno'); plt.axis('off')


In [ ]:
# SCRIPT PER Zippare i risultati
import shutil

# Assicurati di usare i nomi delle cartelle corrette
output_zip_filename = "Radio_Maschere_3B"
dir_to_zip = "/content/Radio_Masks_Final_v5"

shutil.make_archive(output_zip_filename, 'zip', dir_to_zip)
print(f"✅ Creato file '{output_zip_filename}.zip' con le maschere numeriche.")

output_zip_filename_qa = "Radio_QA_3B"
dir_to_zip_qa = "/content/Radio_QA_Final_v5"

shutil.make_archive(output_zip_filename_qa, 'zip', dir_to_zip_qa)
print(f"✅ Creato file '{output_zip_filename_qa}.zip' con le immagini di controllo.")